<a href="https://colab.research.google.com/github/thisisanu/online_deep_learning/blob/main/homework3/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.cuda.is_available()
!nvidia-smi
!pip install numpy torch torchvision tqdm matplotlib

Mon Nov 10 08:04:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
import os
import shutil

# Ensure we are in /content
os.chdir("/content")
print("Current directory:", os.getcwd())

# If a folder named homework2 exists, remove it completely
if os.path.exists("online_deep_learning"):
    shutil.rmtree("online_deep_learning")
    print("Old online_deep_learning folder removed.")
else:
    print("No existing online_deep_learning folder found.")

# Remove the .git folder to disconnect from the current repository
!rm -rf .git
!git clone https://github.com/thisisanu/online_deep_learning.git
%cd online_deep_learning/homework3


Current directory: /content
Old online_deep_learning folder removed.
Cloning into 'online_deep_learning'...
remote: Enumerating objects: 771, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (242/242), done.
remote: Total 771 (delta 179), reused 4 (delta 2), pack-reused 527 (from 6)
Receiving objects: 100% (771/771), 3.36 MiB | 8.77 MiB/s, done.
Resolving deltas: 100% (515/515), done.
/content/online_deep_learning/homework3


In [14]:

#download the datasets by running the following command from the main directory
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/classification_data.zip -o ./classification_data.zip && unzip -qo classification_data.zip
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/drive_data.zip -o ./drive_data.zip && unzip -qo drive_data.zip

In [15]:
import os
# Paths
train_path = './classification_data/train'
val_path = './classification_data/val'

def print_data_overview(directory, name):
    print(f"\n{name} ({directory}):")
    if os.path.exists(directory):
        items = os.listdir(directory)
        print(f"Total items: {len(items)}")
    else:
        print("Directory does not exist!")

# Print directories
print_data_overview(train_path, "Training data")
# List train folders
train_folders = os.listdir(train_path)
print("Train folders:")
print(train_folders[:10])  # show first 10
!ls classification_data/train/*.csv
!head classification_data/train/labels.csv

print_data_overview(val_path, "Validation data")
# List val folders
val_folders = os.listdir(val_path)
print("Validation folders:")
print(val_folders[:10])  # show first 10
!ls classification_data/val/*.csv
!head classification_data/val/labels.csv


Training data (./classification_data/train):
Total items: 21001
Train folders:
['16217.jpg', '11200.jpg', '13724.jpg', '04246.jpg', '11430.jpg', '01452.jpg', '13037.jpg', '18763.jpg', '04703.jpg', '17246.jpg']
classification_data/train/labels.csv
file,label,track
00001.jpg,background,abyss
00002.jpg,background,abyss
00003.jpg,background,abyss
00004.jpg,background,abyss
00005.jpg,background,abyss
00006.jpg,background,abyss
00007.jpg,background,abyss
00008.jpg,background,abyss
00009.jpg,background,abyss

Validation data (./classification_data/val):
Total items: 9001
Validation folders:
['04246.jpg', '01452.jpg', '04703.jpg', '00998.jpg', '05644.jpg', '06780.jpg', '05532.jpg', '05614.jpg', '01497.jpg', '07837.jpg']
classification_data/val/labels.csv
file,label,track
00001.jpg,background,candela_city
00002.jpg,background,candela_city
00003.jpg,background,candela_city
00004.jpg,background,candela_city
00005.jpg,background,candela_city
00006.jpg,background,candela_city
00007.jpg,background,

**Reasoning**:
The previous steps successfully created the `train_classification.py` file. Now, to verify its functionality and ensure the initial setup components (device config, data transforms, data loading) are correctly integrated, I will execute the script. This will confirm that the file is executable and performs the expected setup operations.



In [ ]:
import sys
import os

print("Verifying train_classification.py setup...")
!python -m homework.train_classification
print("Verification complete.")

Verifying train_classification.py setup...
Using device: cuda
[train_classification] Looking for labels.csv at: ./classification_data/train/labels.csv
[get_class_names] Looking for labels.csv at: ./classification_data/train/labels.csv
Found classes: ['background', 'bomb', 'kart', 'nitro', 'pickup', 'projectile']

Epoch 1/10
--------------------


In [12]:
import sys
import os

print("Verifying train_detection.py setup...")
!python -m homework.train_detection

print("Verification complete.")

Verifying train_detection.py setup...
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/online_deep_learning/homework3/homework/train_detection.py", line 6, in <module>
    from datasets.road_dataset import load_data
ModuleNotFoundError: No module named 'datasets.road_dataset'
Verification complete.


In [37]:
%ls
!git status

# Be careful not to "git add *" since there are datasets and logs
!git add homework/*.py
!git add *.py
!git commit -m "update"
!git push origin main

ada3488.zip              drive_data/     Homework3.ipynb
bundle.py                drive_data.zip  README.md
classification_data/     grader/         requirements.txt
classification_data.zip  homework/
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	ada3488.zip
	classification_data.zip
	classification_data/
	drive_data.zip
	drive_data/
	grader/__pycache__/
	grader/datasets/__pycache__/
	homework/__pycache__/
	homework/classifier.th
	homework/datasets/__pycache__/

nothing added to commit but untracked files present (use "git add" to track)
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@32bd00ffe7c6.(none)')
fatal: could not read U

In [35]:
import os
zip_file = 'ada3488.zip'
if os.path.exists(zip_file):
    os.remove(zip_file)
    print(f"Removed existing {zip_file}")
else:
    print(f"{zip_file} not found, no need to remove.")

Removed existing ada3488.zip


In [7]:
!pwd
!python3 bundle.py homework ada3488
!python3 -m grader ada3488.zip -vv --disable_color


/content/online_deep_learning/homework3
metrics.py
train_classification.py
train_detection.py
classifier.th
datasets
models.py
__init__.py
datasets/road_dataset.py
datasets/road_utils.py
datasets/road_transforms.py
datasets/classification_dataset.py
datasets/__init__.py
Submission created: /content/online_deep_learning/homework3/ada3488.zip 15.84 MB
Public grader loaded.
[DEBUG    00:00:000] Loading assignment
[DEBUG    00:00:120] Loading grader
[INFO     00:00:120] Classifier
[INFO     00:00:538]   - Predict                                            [ 10 / 10 ]
[WARNING  00:04:268]   - Accuracy                                           [ 0 / 25 ]
[WARNING  00:04:269] accuracy: 0.454, required > 0.8
[WARNING  00:04:269]   - Accuracy: Extra Credit                             [ 0 / 2 ]
[INFO     00:04:270]  --------------------------------------------------    [  10 /  35 ]
[INFO     00:04:271] Detector
[DEBUG    00:04:306] Loaded 2000 samples from 4 episodes
[INFO     00:04:318]   - Pr

In [40]:
print("Running the grader again with the updated zip...")
!python3 -m grader ada3488.zip -vv --disable_color
print("Grader execution complete.")

Running the grader again with the updated zip...
Public grader loaded.
[DEBUG    00:00:000] Loading assignment
[DEBUG    00:00:121] Loading grader
[INFO     00:00:121] Classifier
[INFO     00:00:523]   - Predict                                            [ 10 / 10 ]
[WARNING  00:04:266]   - Accuracy                                           [ 0 / 25 ]
[WARNING  00:04:266] accuracy: 0.453, required > 0.8
[WARNING  00:04:267]   - Accuracy: Extra Credit                             [ 0 / 2 ]
[INFO     00:04:269]  --------------------------------------------------    [  10 /  35 ]
[INFO     00:04:270] Detector
[DEBUG    00:04:305] Loaded 2000 samples from 4 episodes
[INFO     00:04:318]   - Predict                                            [ 10 / 10 ]
[WARNING  00:04:318]   - Segmentation Accuracy                              [ 0 / 10 ]
[WARNING  00:04:318] detector.th not found
[WARNING  00:04:319]   - Segmentation IoU                                   [ 0 / 25 ]
[WARNING  00:04:319] dete